In [1]:
import os
import json
import time
from itertools import product

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import xgboost as xgb

from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from tabulate import tabulate

import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler



In [2]:
# Importamos datos depurados
df = pd.read_csv('../../Data/dropout_depurado.csv', sep=',')

# Separamos features de target
y = df['Dropout']
X = df.drop('Dropout', axis=1)

# Separamos train de test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.25)


Vamos a crear un excel de resultados en el que almacenemos los resultados de cada modelo entrenado: la clase, la parametrizacion, la matriz de confusion en 4 columnas (TP, FP, FN, TN), y las variables input del modelo (si en el futuro el pretratamiento cambia, seremos capaces de rastrearlo y comparar distintos pretratamientos)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict, StratifiedKFold

# Crear un validador cruzado estratificado para conservar la proporción de clases en cada pliegue
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Importamos el excel creado para guardar los resultados
operacion_resultados = pd.read_excel('tabla_resultados.xlsx')

# Definimos la lista de modelos a probar
models = [
    DecisionTreeClassifier(),
    LogisticRegression(max_iter=999999),
    RandomForestClassifier(),
    SVC(),
    XGBClassifier(),
    KNeighborsClassifier(),
    GaussianNB(),  # Naive Bayes Classifier
    AdaBoostClassifier(),  # AdaBoost Classifier
    GradientBoostingClassifier(),  # Gradient Boosting Classifier
    MLPClassifier()  # Multi-layer Perceptron Classifier
]

# Función para mostrar métricas y matriz de confusión
def print_evaluation_metrics(model_name, y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    
    print(f"Modelo: {model_name}")
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, 
                annot_kws={"size": 10}, linewidths=0.5, 
                xticklabels=['No Fraude', 'Fraude'], 
                yticklabels=['No Fraude', 'Fraude'])
    plt.xlabel('Predicción')
    plt.ylabel('Verdadero')
    plt.title(f'Matriz de Confusión - {model_name}')
    plt.show()
    
    return cm

# Entrenar y evaluar cada modelo con validación cruzada
for model in models:
    model_name = model.__class__.__name__
    
    # Crear el pipeline con MinMaxScaler y el modelo actual
    pipeline = Pipeline([
        ('scaler', MinMaxScaler()),  # Escalador MinMaxScaler
        ('model', model)  # Modelo a entrenar
    ])
    
    # Obtener predicciones de validación cruzada usando el pipeline
    y_pred_cv = cross_val_predict(pipeline, X_train, y_train, cv=cv)
    
    # Obtener las métricas
    cm = print_evaluation_metrics(f'{model_name} (Validación Cruzada)', y_train, y_pred_cv)
    
    # Capturando detalles del modelo
    params = model.get_params() if hasattr(model, 'get_params') else 'default'
    params_json = json.dumps(params)
    input_variables = list(X_train.columns)
    execution_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Extraer TP, FP, FN, TN de la matriz de confusión
    tn, fp, fn, tp = cm.ravel()
    
    # Crear un nuevo DataFrame con los resultados
    new_row = pd.DataFrame({
        'Modelo': [model_name],
        'Comentarios': "Valores Latex Cuadro 1",
        'TP': [tp],
        'FP': [fp],
        'FN': [fn],
        'TN': [tn],
        'Parametrizacion': [params_json],
        'Variables input del modelo': [input_variables],
        'Fecha de ejecucion del modelo': [execution_date]
    })
    
    # Concatenar el nuevo DataFrame con el DataFrame existente
    operacion_resultados = pd.concat([operacion_resultados, new_row], ignore_index=True)

    # Guardar el DataFrame de resultados actualizados
    operacion_resultados.to_excel('tabla_resultados.xlsx', index=False)


-------------------------------------

Esto es para hacer la burrada de los 62k modelos de XGBoost

In [4]:
model = XGBClassifier()

# Define the expanded parameter grid for XGBClassifier
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01],
    'max_depth': [None, 3],
    'min_child_weight': [1, 3],
    'subsample': [0.9, 1.0],
    'colsample_bytree': [0.9, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5],
    'scale_pos_weight': [1, 2, 5],
    'objective': ['binary:logistic', 'binary:hinge', 'binary:logitraw'],
    'booster': ['gbtree', 'dart'],
    'max_delta_step': [0, 1, 5],
    'eval_metric': ['logloss', 'auc']
}

# Generate all combinations of parameters
param_combinations = list(product(*param_grid.values()))

# Ruta del archivo
file_path = 'tabla_resultados.xlsx'

# Comprobar si el archivo existe
if os.path.exists(file_path):
    # Leer el archivo
    operacion_resultados = pd.read_excel(file_path)
else:
    # Inicializar el DataFrame vacío
    operacion_resultados = pd.DataFrame(columns=['Modelo', 'Parametrizacion', 'TP', 'FP', 'FN', 'TN', 'Variables input del modelo', 'Fecha de ejecucion del modelo', 'Comentarios'])


In [ ]:
from datetime import datetime
import time
import cupy as cp

# Iniciar el tiempo al inicio del bucle
start_time = time.time()

# Pedir al usuario que especifique desde qué valor de i comenzar
# start_i = int(input("Ingrese el valor inicial de i: ")) # Esto es en caso de que se desee detener el entrenamiento y reiniciar desde el mismo punto
start_i = 0

# Iterar sobre todas las combinaciones de parámetros
for i, combination in enumerate(param_combinations[start_i:], start=start_i):
    # Crear un diccionario de parámetros para el modelo
    params = {key: combination[j] for j, key in enumerate(param_grid.keys())}

    print(f'Probando con parametrizacion {i+1}/{len(param_combinations)} con parametros: {params}.')

    # Crear y entrenar el modelo con esta configuración
    model = XGBClassifier(
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        min_child_weight=params['min_child_weight'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        gamma=params['gamma'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        scale_pos_weight=params['scale_pos_weight'],
        objective=params['objective'],
        booster=params['booster'],
        max_delta_step=params['max_delta_step'],
        tree_method='hist',
        device='cuda',
        use_label_encoder=False,
        eval_metric=params['eval_metric'], 
        predictor='gpu_predictor'
    )

    model.fit(np.array(X_train), np.array(y_train))

    # Predecir los valores de test
    y_pred = model.predict(np.array(X_test))

    # Calcular la matriz de confusión
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(conf_matrix)
    print('\n' + '-'*40 + '\n')

    # Añadir los valores de la nueva parametrización a la lista de resultados
    new_row = {
        'Modelo': 'XGBClassifier',
        'Parametrizacion': params,
        'TP': conf_matrix[1][1],
        'FP': conf_matrix[0][1],
        'FN': conf_matrix[1][0],
        'TN': conf_matrix[0][0],
        'Variables input del modelo': list(X_train.columns),
        'Fecha de ejecucion del modelo': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'Comentarios': 'Entrenamiento real XGBoost'
    }

    # Concatenar el nuevo DataFrame con el DataFrame existente
    operacion_resultados = pd.concat([operacion_resultados, pd.DataFrame([new_row])], ignore_index=True)

    # Guardar en el archivo Excel cada 100 iteraciones
    if (i + 1) % 10 == 0:
        operacion_resultados.to_excel(file_path, index=False)
        print(f"Guardado automático en el archivo Excel después de {i + 1} iteraciones.")

# Guardar el DataFrame actualizado en el archivo Excel al finalizar todas las iteraciones
operacion_resultados.to_excel(file_path, index=False)

end_time = time.time()
print(f"Grid search completed in {end_time - start_time} seconds.")


---------------------------------------------

Esto es para estudiar los 3 mejores modelos segun el codigo de antes.

In [3]:
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'subsample': [0.8, 1.0],
    'max_features': [None, 'sqrt', 'log2'],
}

param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'class_weight': [None, 'balanced'],
}

param_grid_lr = {
    'C': [0.1, 1, 10],
    'penalty': ['l2'],
    'class_weight': [None, 'balanced'],
    'solver': ['lbfgs', 'liblinear'],
}


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from itertools import product
from datetime import datetime
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# Ruta del archivo
file_path = 'tabla_resultados.xlsx'

# Cargar archivo si ya existe
if os.path.exists(file_path):
    operacion_resultados = pd.read_excel(file_path)
else:
    operacion_resultados = pd.DataFrame(columns=['Modelo', 'Parametrizacion', 'TP', 'FP', 'FN', 'TN', 'Variables input del modelo', 'Fecha de ejecucion del modelo', 'Comentarios'])

# Diccionario de modelos y grids
model_grid_dict = {
    'GradientBoostingClassifier': (GradientBoostingClassifier(), param_grid_gb),
    'SVC': (SVC(), param_grid_svc),
    'LogisticRegression': (LogisticRegression(), param_grid_lr),
}

# Validación cruzada estratificada (3 folds)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Bucle que recorre cada modelo
for model_name, (model, param_grid) in tqdm(model_grid_dict.items(), desc="Modelos", unit="modelo"):
    # Obtener combinaciones de parámetros
    param_combinations = list(product(*param_grid.values()))

    # Bucle que recorre las combinaciones de parámetros con barra de progreso
    for i, combination in enumerate(tqdm(param_combinations, desc=f"{model_name} Grid Search", unit="combinación")):
        # Crear diccionario de parámetros
        params = {key: combination[j] for j, key in enumerate(param_grid.keys())}
        
        print(f'Ejecutando {model_name} con configuración {i+1}/{len(param_combinations)}: {params}')

        # Crear el pipeline con MinMaxScaler y el modelo actual
        pipeline = Pipeline([
            ('scaler', MinMaxScaler()),  # Escalador MinMaxScaler para los campos numéricos
            ('model', model.set_params(**params))  # Modelo con los parámetros configurados
        ])

        # Obtener predicciones con validación cruzada
        y_pred_cv = cross_val_predict(pipeline, np.array(X_train), np.array(y_train), cv=cv)

        # Calcular la matriz de confusión
        conf_matrix = confusion_matrix(y_train, y_pred_cv)
        # print(conf_matrix)

        # Guardar resultados
        new_row = {
            'Modelo': model_name,
            'Parametrizacion': params,
            'TP': conf_matrix[1][1],
            'FP': conf_matrix[0][1],
            'FN': conf_matrix[1][0],
            'TN': conf_matrix[0][0],
            'Variables input del modelo': list(X_train.columns),
            'Fecha de ejecucion del modelo': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            'Comentarios': f'Entrenamiento {model_name} con validación cruzada de 3 folds'
        }

        operacion_resultados = pd.concat([operacion_resultados, pd.DataFrame([new_row])], ignore_index=True)

        # Guardar en Excel cada 10 iteraciones
        if (i + 1) % 10 == 0:
            operacion_resultados.to_excel(file_path, index=False)
            print(f'Guardado automático después de {i + 1} iteraciones.')

# Guardar los resultados finales
operacion_resultados.to_excel(file_path, index=False)


Esto es para generar la tabla de interpretabilidad

In [8]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Parámetros del mejor modelo de logistic regresion chavales
params = {'C': 1, 'penalty': 'l2', 'class_weight': None, 'solver': 'lbfgs', 'max_iter': 999999}

# Entrenar el modelo con los parámetros dados
log_reg_model = LogisticRegression(**params)
log_reg_model.fit(X_train, y_train)

# Obtener los coeficientes
coef = log_reg_model.coef_[0]
intercept = log_reg_model.intercept_[0]

# Crear un DataFrame para los coeficientes
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': coef
})

# Calcular los odds ratios
coef_df['Odds Ratio'] = np.exp(coef_df['Coefficient'])

# Suponiendo que 'Feature_1' es una variable categórica para la cual calcularemos WOE
# Nota: Debes reemplazar 'Feature_1' con el nombre de la característica real en tu DataFrame
if 'Feature_1' in X_train.columns:
    # Calcular la proporción de eventos y no eventos
    event_rate = y_train.mean()  # Proporción de eventos
    non_event_rate = 1 - event_rate  # Proporción de no eventos

    # Calcular WOE para cada categoría en 'Feature_1'
    feature_1_df = pd.concat([X_train[['Feature_1']], y_train], axis=1)
    woe_df = feature_1_df.groupby('Feature_1').apply(
        lambda x: pd.Series({
            'Proporción de Eventos': x[y_train == 1].shape[0] / y_train.shape[0],
            'Proporción de No Eventos': x[y_train == 0].shape[0] / y_train.shape[0]
        })
    ).reset_index()

    woe_df['WOE'] = np.log(woe_df['Proporción de Eventos'] / woe_df['Proporción de No Eventos'])
    coef_df = coef_df.merge(woe_df[['Feature_1', 'WOE']], left_on='Feature', right_on='Feature_1', how='left')

    # Limpiar el DataFrame resultante
    coef_df = coef_df.drop(columns=['Feature_1'])

# Imprimir los coeficientes y los odds ratios
print("Coeficientes del modelo de regresión logística:")
print(coef_df)

print("\nIntercepto del modelo:")
print(intercept)

# Evaluar el rendimiento del modelo en el conjunto de entrenamiento
# Predicciones
y_pred = log_reg_model.predict(X_train)

# Imprimir la matriz de confusión
print("\nMatriz de Confusión:")
print(confusion_matrix(y_train, y_pred))

# Imprimir el informe de clasificación
print("\nInforme de Clasificación:")
print(classification_report(y_train, y_pred))


Coeficientes del modelo de regresión logística:
                  Feature  Coefficient  Odds Ratio
0                     Age     0.028480    1.028889
1       Unemployment_rate     0.113560    1.120259
2          Inflation_rate    -0.036043    0.964599
3                     GDP     0.027682    1.028068
4               Avg_grade    -0.016160    0.983970
..                    ...          ...         ...
237              Gender_1     0.218717    1.244479
238  Scholarship_holder_0     0.240485    1.271866
239  Scholarship_holder_1    -0.239497    0.787024
240       International_0     0.127612    1.136112
241       International_1    -0.126624    0.881065

[242 rows x 3 columns]

Intercepto del modelo:
1.2589218173192143

Matriz de Confusión:
[[2135  117]
 [ 270  796]]

Informe de Clasificación:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      2252
           1       0.87      0.75      0.80      1066

    accuracy                     

In [10]:
coef_df.to_excel('coeficientes_mejor_log.xlsx', index=False)